# unittest.mock
## Basic methods

### Create a Mock object

In [2]:
from unittest import mock
m = mock.Mock()

### Set a default return value when calling mock object

In [3]:
m.return_value = 42
m()

42

### Assign different return values for successive calls

In [4]:
m.side_effect = ['foo', 'bar', 'baz']
print(m())
print(m())
print(m())

foo
bar
baz


### Check whether mock object has been called at least once

In [5]:
m = mock.Mock()
m()
m.assert_called()

Returns *None*, if called, otherwise raises *AssertionError*

### Check whether mock object has been called exactly once

In [6]:
m = mock.Mock()
try:
    m.assert_called_once()
except AssertionError:
    print("No, I wasn't called (yet)")
m()
if not m.assert_called_once():
    print("Yeah, now I was called")

No, I wasn't called (yet)
Yeah, now I was called


### Get number of calls to mock object

In [7]:
m = mock.Mock()
m()
m()
m.call_count

2

### Get call arguments of last mock call

In [8]:
m = mock.Mock()
m(1, foo='bar')
m.call_args

call(1, foo='bar')

If mock object hasn't been called yet, *None* is returned.

### Get call arguments of all mock calls

In [9]:
m = mock.Mock()
m()
m(1, foo='bar')
m(4, baz='bar2')
m.call_args_list

[call(), call(1, foo='bar'), call(4, baz='bar2')]

### Reset call previous call (won't change mock configuration)

In [10]:
m = mock.Mock()
m()
m(1, foo='bar')
m(4, baz='bar2')
m.reset_mock()
m.call_args_list

[]

## Patching an import module from a project module in our unit test
### Project module (work.py)

```python
import os

def work_on():
    path = os.getcwd()
    print(f'Working on {path}')
    return path
```

The project module imports *os* and uses its *getcwd()* method , which
we want to mock in our test.

### Defining the unit test

```python
from unittest import TestCase, mock

from work import work_on

class TestWorkMockingModule(TestCase):

    def test_using_context_manager(self):
        with mock.patch('work.os') as mocked_os:
            work_on()
            mocked_os.getcwd.assert_called_once()
```

* The test module imports the project module's method *work_on()*
* At the beginning of the test, the work.os module (which is the os module in
  our work.py module) is patched with [MagickMock][1] object (here called *mocked_os*)
* When the *work_on()* method is called afterward, the *MagicMock* object is
  returned instead of the original module (os)
* The returned *MagicMock* object assigned the attribute name to the function
  that is called on the patched module (here: *os.getcwd()*)

[1]: https://docs.python.org/3/library/unittest.mock.html?highlight=magicmock#unittest.mock.MagicMock

### Executing the test

```python
import unittest
unittest.main(argv=[''], verbosity=2, exit=False)
```

Alternatively, the :ulined:`patch decorator` can be used:

```python
@mock.patch('work.os')
def test_using_decorator(self, mocked_os):
    work_on()
    mocked_os.getcwd.assert_called_once()

```

To define a **return value** for the mocked *os.getcwd()* function, define it
like this:

```python
def test_using_return_value(self):
    """Note 'as' in the context manager is optional"""
    with mock.patch('work.os.getcwd', return_value='testing'):
        assert work_on() == 'testing'
```

Applying a `return_value='testing'` will return the string when
calling the `work.os.getcwd()` function.

Running the test leads to this output:

```python
import unittest
unittest.main()
```
```
Working on testing
```

## Patching classes
To test project classes that interact with other project classes in isolation,
the other project classes must be mocked (in order to determine test failure
to a specific class and method, not it's depending classes).

Example:

### Project module (worker.py)

In this module, the `Worker` class needs to be tested for these two things:

- the `Worker` calls `Helper` with `db`
- the `Worker` returns the expected path supplied by `Helper `

```python
import os


class Helper:

    def __init__(self, path):
        self.path = path

    def get_path(self):
        base_path = os.getcwd()
        return os.path.join(base_path, self.path)


class Worker:

    def __init__(self):
        self.helper = Helper('db')

    def work(self):
        path = self.helper.get_path()
        print(f'Working on {path}')
        return path
```

To test this, the `Worker` needs to be isolated from `Helper`,
so that failures can be associated with the `Worker` class only.

Consequently, the `Helper` class must be patched:

```python
from unittest import TestCase, mock
from worker import Worker, Helper


class TestWorkerModule(TestCase):

    def test_patching_class(self):
        with mock.patch('worker.Helper') as MockHelper:
            MockHelper.return_value.get_path.return_value = 'testing'
            worker = Worker()
            MockHelper.assert_called_once_with('db')
            self.assertEqual(worker.work(), 'testing')
```

Here


*NEEDS TO BE CONTINUED...*